In [1]:
import pink_utils as pu
import pandas as pd
import numpy as np
import time

In [2]:
hm = pu.heatmap('../Experiment/Experiment_F3W2_95_5/F3W2_95_5_Testing_Images_EDMatrix.bin')
images = pu.image_binary('../Experiment/Experiment_F3W2_95_5/F3W2_95_5_Testing_Images.bin')

In [3]:
df = pd.read_csv('../Experiment/Experiment_F3W2_95_5/RGZ_Test_Images_Dataframe.csv')

In [4]:
df.shape

(7464, 78)

In [5]:
images.file_head

(7464, 2, 167, 167)

In [6]:
hm.file_head

(7464, 15, 15, 1)

In [7]:
before = time.time()
print(f'Before time is {before}')
print('Loading the heatmaps')
maps = [hm.ed(index=i, prob=True) for i in range(hm.file_head[0])]

# print('Loading the FIRST pre-processed images')
# first = [images.get_image(index=i, channel=0) for i in range(hm.file_head[0])]

# print('Loading the WISE pre-processed images')
# wise = [images.get_image(index=i, channel=1) for i in range(hm.file_head[0])]

print('Loading the cube pre-processed images')
cubes = [np.concatenate([images.get_image(index=i, channel=0).flatten(), images.get_image(index=i, channel=1).flatten()]) for i in range(hm.file_head[0])]

after = time.time()
print(f'After time is {after}')
print(f'\tTime difference is {after-before}')


Before time is 1538463340.5563118
Loading the heatmaps
Loading the cube pre-processed images
After time is 1538463420.2692976
	Time difference is 79.71298575401306


In [8]:
import matplotlib.pyplot as plt

In [9]:
from collections import defaultdict

In [10]:
book = defaultdict(list)

for c, row in df.iterrows():
    book[row['label']].append(maps[c])

In [13]:
from sklearn.ensemble.forest import RandomForestRegressor
# from sklearn import cross_validation
from sklearn import model_selection

In [14]:
np.array(cubes).shape

(7464, 55778)

In [15]:
np.random.seed(7464)
index = np.random.permutation(len(maps))

labels = df['label']
num_comp  = np.array([int(i.split('_')[0]) for i in labels])[index]
num_peaks = np.array([int(i.split('_')[1]) for i in labels])[index]

# Keep heat maps to ensure nothing else breaks
heatmaps  = np.array(cubes)[index]

total = heatmaps.shape[0]

suf_df = df.iloc[index]


In [ ]:
before = time.time()
print(f'Before time is {before}')

n_estimators = 64 # how many trees, should be 256
cores = 8 # number of cpu cores to speed it up

peaks_rf = RandomForestRegressor(n_estimators=n_estimators, n_jobs=cores, bootstrap=True)
comp_rf  = RandomForestRegressor(n_estimators=n_estimators, n_jobs=cores, bootstrap=True)

peaks_skf = model_selection.StratifiedKFold(n_splits=5, shuffle=False)
comp_skf = model_selection.StratifiedKFold(n_splits=5, shuffle=False)

print('Peaks training')
peaks_res = model_selection.cross_val_predict(peaks_rf, heatmaps, num_peaks, cv=peaks_skf, n_jobs=cores)

print('Comp training')
comp_res = model_selection.cross_val_predict(comp_rf, heatmaps, num_comp, cv=comp_skf, n_jobs=cores)

after = time.time()
print(f'After time is {after}')
print(f'\tTime difference is {after-before}')


Before time is 1538463584.934784
Peaks training


In [ ]:
fig, ax = plt.subplots(1,1)

ax.scatter(comp_res, peaks_res, alpha=0.3)

ax.set(xlabel='Predicted Components',
       ylabel='Predicted Peaks')

fig.show()

In [ ]:
fig, ax = plt.subplots(1,1)

hb = ax.hexbin(comp_res, peaks_res, 
               bins='log', gridsize=20, mincnt=1)

cb = fig.colorbar(hb, ax=ax)
cb.set_label('Log(N+1)')

ax.set(xlabel='Predicted Components',
       ylabel='Predicted Peaks')

# fig.show()
fig.savefig('Images/Predicted_Features_Heatmap.pdf')

In [ ]:
from mpl_toolkits.axes_grid1 import make_axes_locatable

def subplot_cbar(fig, ax, im):
        divider = make_axes_locatable(ax)
        cax0 = divider.append_axes('right', size='5%', pad=0.05)
        fig.colorbar(im, cax=cax0)
    

fig, axes = plt.subplots(2,3, figsize=(10, 5))

for c, l in enumerate(np.sort(labels[index].unique())):
    mask = l == labels[index]
    
    ax = axes.flatten()[c]
    im = ax.hexbin(comp_res[mask], peaks_res[mask], gridsize=20, mincnt=1)
    ax.set(title=f'{l} - N = {np.sum(mask)}', xlim=[1,3],
          xlabel='Redicted Components',
          ylabel='Predicted Peaks')
    
    subplot_cbar(fig, ax, im)
    
fig.tight_layout()
fig.show()
fig.savefig('Images/Predicted_Features_Heatmap_Classes.pdf')

In [ ]:
fig, (ax1,ax2) = plt.subplots(1,2, figsize=(10,5))

ax1.hist( comp_res - num_comp, bins=250 )
ax1.set(xlabel='Predicted components - RGZ classified components',
        ylabel='Counts')

ax2.hist( peaks_res - num_peaks, bins=250 )
ax2.set(xlabel='Predicted peaks - RGZ classified peaks',
        ylabel='Counts')


# fig.show()
fig.savefig('Images/Predicted_Featurs_Histogram.pdf')

In [ ]:
fig, (ax1,ax2) = plt.subplots(1,2, figsize=(10,5))

val = comp_res - num_comp
ax1.hist( val[val!=0], bins=250 )
ax1.set(xlabel='Predicted components - RGZ classified components',
        ylabel='Counts')

val = peaks_res - num_peaks
ax2.hist(val[val != 0], bins=250 )
ax2.set(xlabel='Predicted peaks - RGZ classified peaks',
        ylabel='Counts')


# fig.show()
fig.savefig('Images/Predicted_Featurs_Histogram_No_Zero.pdf')

In [ ]:
bin_df = pd.cut(suf_df['consensus.ir_level'] , [0.6, 0.7, 0.8, 0.9, 1.0], labels=['0.6-0.7','0.7-0.8','0.8-0.9','0.9-1.0',])

fig, (ax1,ax2) = plt.subplots(1,2, figsize=(8,5))
linestyles = [':','--','-.','-']

for c, b in enumerate(np.sort(bin_df.unique())):
    print(b)
    mask = b == bin_df
    
    val = comp_res - num_comp
    ax1.hist( val[(mask)&(val!=0)], bins=50, label=f'{b}', histtype='step', linestyle=linestyles[c])
    
    val = peaks_res - num_peaks
    ax2.hist(val[(mask)&(val!=0)], bins=50, label=f'{b}', histtype='step', linestyle=linestyles[c])

ax1.set(xlabel='Predicted components - RGZ classified components',
            ylabel='Counts')
ax2.set(xlabel='Predicted peaks - RGZ classified peaks',
        ylabel='Counts')

ax1.legend(title='Radio Consensus')
ax2.legend(title='Radio Consensus')
fig.tight_layout()
fig.show()



In [ ]:
bin_df = pd.cut(suf_df['consensus.ir_level'] , [0.6, 0.7, 0.8, 0.9, 1.0], labels=['0.6-0.7','0.7-0.8','0.8-0.9','0.9-1.0',])

fig, (ax1,ax2) = plt.subplots(1,2, figsize=(8,5))
linestyles = [':','--','-.','-']

for c, b in enumerate(np.sort(bin_df.unique())):
    print(b)
    mask = b == bin_df
    
    val = comp_res - num_comp
    ax1.hist( val[mask], bins=50, label=f'{b}', histtype='step', linestyle=linestyles[c])
    
    val = peaks_res - num_peaks
    ax2.hist(val[mask], bins=50, label=f'{b}', histtype='step', linestyle=linestyles[c])

ax1.set(xlabel='Predicted components - RGZ classified components',
            ylabel='Counts')
ax2.set(xlabel='Predicted peaks - RGZ classified peaks',
        ylabel='Counts')

ax1.legend(title='Radio Consensus')
ax2.legend(title='Radio Consensus')
fig.tight_layout()
fig.show()


In [ ]:
fig, (ax1,ax2) = plt.subplots(1,2, figsize=(9,5))
linestyles = [':','--','-.','-']

val = comp_res - num_comp

bin_df = pd.cut(suf_df['consensus.radio_level'] , [0.6, 0.7, 0.8, 0.9, 1.0], labels=['0.6-0.7','0.7-0.8','0.8-0.9','0.9-1.0',])
for c, b in enumerate(np.sort(bin_df.unique())):
    mask = b == bin_df    
    ax1.hist( val[(mask)&(val!=0)], bins=50, label=f'{b}', histtype='step', linestyle=linestyles[c])

    
bin_df = pd.cut(suf_df['consensus.ir_level'] , [0.6, 0.7, 0.8, 0.9, 1.0], labels=['0.6-0.7','0.7-0.8','0.8-0.9','0.9-1.0',])
for c, b in enumerate(np.sort(bin_df.unique())):
    mask = b == bin_df    
    ax2.hist( val[(mask)&(val!=0)], bins=50, label=f'{b}', histtype='step', linestyle=linestyles[c])

ax1.set(xlabel='Predicted components - RGZ classified components',
            ylabel='Counts')
ax2.set(xlabel='Predicted components - RGZ classified components',
        ylabel='Counts')

ax1.legend(title='Radio Consensus')
ax2.legend(title='IR Consensus')
fig.tight_layout()
# fig.show()
fig.savefig('Images/Components_Histogram_Consensus.pdf')


In [ ]:
fig, (ax1,ax2) = plt.subplots(1,2, figsize=(8,5))
linestyles = [':','--','-.','-']

val = peaks_res - num_peaks

bin_df = pd.cut(suf_df['consensus.radio_level'] , [0.6, 0.7, 0.8, 0.9, 1.0], labels=['0.6-0.7','0.7-0.8','0.8-0.9','0.9-1.0',])
for c, b in enumerate(np.sort(bin_df.unique())):
    mask = b == bin_df    
    ax1.hist( val[(mask)&(val!=0)], bins=50, label=f'{b}', histtype='step', linestyle=linestyles[c])

    
bin_df = pd.cut(suf_df['consensus.ir_level'] , [0.6, 0.7, 0.8, 0.9, 1.0], labels=['0.6-0.7','0.7-0.8','0.8-0.9','0.9-1.0',])
for c, b in enumerate(np.sort(bin_df.unique())):
    mask = b == bin_df    
    ax2.hist( val[(mask)&(val!=0)], bins=50, label=f'{b}', histtype='step', linestyle=linestyles[c])

ax1.set(xlabel='Predicted peaks - RGZ classified peaks',
            ylabel='Counts')
ax2.set(xlabel='Predicted peaks - RGZ classified peaks',
        ylabel='Counts')

ax1.legend(title='Radio Consensus')
ax2.legend(title='IR Consensus')
fig.tight_layout()
# fig.show()
fig.savefig('Images/Peaks_Histogram_Consensus.pdf')

In [ ]:
fig, (ax1,ax2) = plt.subplots(1,2, figsize=(8,5))
linestyles = [':','--','-.','-', '-',':']

val = comp_res - num_comp

bin_df = np.unique(suf_df['label'])
for c, b in enumerate(np.sort(bin_df)):
    mask = b ==  suf_df['label']   
    ax1.hist( val[(mask)&(val!=0)], bins=25, label=f'{b}', histtype='step', linestyle=linestyles[c])


val = peaks_res - num_peaks


bin_df = np.unique(suf_df['label'])
for c, b in enumerate(np.sort(bin_df)):
    mask = b == suf_df['label']    
    ax2.hist( val[(mask)&(val!=0)], bins=25, label=f'{b}', histtype='step', linestyle=linestyles[c])

ax1.set(xlabel='Predicted components - RGZ classified components',
            ylabel='Counts')
ax2.set(xlabel='Predicted peaks - RGZ classified peaks',
        ylabel='Counts')

ax1.legend(title='RGZ Label')
ax2.legend(title='RGZ Label')
fig.tight_layout()
# fig.show()
fig.savefig('Images/RGZ_Label_Predicted_Features.pdf')

In [ ]:
bin_df = np.unique(suf_df['label'])
linestyles = [':','--','-.','-', '-',':']

fig, axes = plt.subplots(len(bin_df), 2, figsize=(8, len(bin_df)*4))

for count, l in enumerate(np.sort(bin_df)):
    ax1, ax2 = axes[count]
    
    l_mask = l == suf_df['label']
    val_peak = peaks_res - num_peaks
    val_comp = comp_res - num_comp 
    
    cen_df = pd.cut(suf_df['consensus.radio_level'] , [0.6, 0.7, 0.8, 0.9, 1.0], labels=['0.6-0.7','0.7-0.8','0.8-0.9','0.9-1.0',])
    for c, b in enumerate(np.sort(cen_df.unique())):
        mask = b == cen_df    
        ax1.hist( val_peak[(mask)&(val_peak!=0)&(l_mask)], bins=50, label=f'{b}', histtype='step', linestyle=linestyles[c])
        ax1.legend(title=f'{l} Consensus')
        ax1.set(xlim=[-2,2], xlabel='Predicted peaks - RGZ peaks')
        
        ax2.hist( val_comp[(mask)&(val_comp!=0)&(l_mask)], bins=50, label=f'{b}', histtype='step', linestyle=linestyles[c])
        ax2.legend(title=f'{l} Consensus')
        ax2.set(xlim=[-2,2], xlabel='Predicted comp - RGZ comp')

fig.tight_layout()
# fig.show()
fig.savefig('Images/Figure_Label_Consensus_Histogram.pdf')

In [ ]:
bin_df

In [ ]:
bin_df = np.unique(suf_df['label'])
linestyles = [':','--','-.','-', '-',':']

fig, axes = plt.subplots(len(bin_df), 2, figsize=(8, len(bin_df)*4))

for count, l in enumerate(np.sort(bin_df)):
    ax1, ax2 = axes[count]
    
    l_mask = l == suf_df['label']
    val_peak = peaks_res - num_peaks
    val_comp = comp_res - num_comp 
    
    cen_df = pd.cut(suf_df['consensus.radio_level'] , [0.6, 0.7, 0.8, 0.9, 1.0], labels=['0.6-0.7','0.7-0.8','0.8-0.9','0.9-1.0',])
    for c, b in enumerate(np.sort(cen_df.unique())):
        mask = b == cen_df    
        ax1.hist( val_peak[(mask)&(l_mask)], bins=50, label=f'{b}', histtype='step', linestyle=linestyles[c])
        ax1.legend(title=f'{l} Consensus')
        ax1.set(xlim=[-2,2], xlabel='Predicted peaks - RGZ peaks')
        
        ax2.hist( val_comp[(mask)&(l_mask)], bins=50, label=f'{b}', histtype='step', linestyle=linestyles[c])
        ax2.legend(title=f'{l} Consensus')
        ax2.set(xlim=[-2,2], xlabel='Predicted comp - RGZ comp')

fig.tight_layout()
# fig.show()
fig.savefig('Images/Figure_Label_Consensus_Histogram_With_Zero.pdf')

In [ ]:
bin_df = np.unique(suf_df['label'])
linestyles = [':','--','-.','-', '-',':', '--']

fig, axes = plt.subplots(len(bin_df), 2, figsize=(8, len(bin_df)*4))

for count, l in enumerate(np.sort(bin_df)):
    ax1, ax2 = axes[count]
    
    l_mask = l == suf_df['label']
    val_peak = peaks_res - num_peaks
    val_comp = comp_res - num_comp 
    
    cen_df = pd.cut(suf_df['consensus.radio_level'] , [0.6, 0.7, 0.8, 0.9, 0.95, 1.0], labels=['0.6-0.7','0.7-0.8','0.8-0.9','0.9-0.95', '0.95-1.0',])
    for c, b in enumerate(np.sort(cen_df.unique())):
        mask = b == cen_df    
        ax1.hist( val_peak[(mask)&(val_peak!=0)&(l_mask)], bins=50, label=f'{b}', histtype='step', linestyle=linestyles[c])
        ax1.legend(title=f'{l} Consensus')
        ax1.set(xlim=[-2,2], xlabel='Predicted peaks - RGZ peaks')
        
        ax2.hist( val_comp[(mask)&(val_comp!=0)&(l_mask)], bins=50, label=f'{b}', histtype='step', linestyle=linestyles[c])
        ax2.legend(title=f'{l} Consensus')
        ax2.set(xlim=[-2,2], xlabel='Predicted comp - RGZ comp')

fig.tight_layout()
# fig.show()